# Modules

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import shapely.geometry as shpg
import matplotlib.pyplot as plt
from matplotlib import cm as colormap
import shapely.ops
from salem import datasets
from cleo import Map
from matplotlib.backends.backend_pdf import PdfPages
%pylab inline

# Files

In [ ]:
wgms_dir = '..\\DOI-WGMS-FoG-2014-09'
output_dir = '..\\LECLERCQ_LINKS'
f_A = os.path.join(wgms_dir, 'WGMS-FoG-2014-09-A-GENERAL-INFORMATION.csv')
f_EE = os.path.join(wgms_dir, 'WGMS-FoG-2014-09-EE-MASS-BALANCE.csv')
f_lec_prop = '..\\glacier_properties.txt'
rgi_dir = '..\\rgi50\\11_rgi50_CentralEurope'
f_rgi = os.path.join(rgi_dir, '11_rgi50_CentralEurope.shp')

# Haversine function

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between one point 
    on the earth and an array of points (specified in decimal degrees)
    """
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371000 # Radius of earth in meters
    return c * r

# Read files

In [ ]:
pda = pd.read_csv(f_A, encoding='iso8859_15')
pda.head(1)

In [ ]:
pdee = pd.read_csv(f_EE, encoding='iso8859_15')
pdee.head(1)

In [ ]:
lec = pd.read_csv(f_lec_prop, sep='\t', na_values='-99999')
lec = lec.rename(columns=lambda x: x.strip())
lec.head(1)

In [ ]:
pdrgi = gpd.read_file(f_rgi)
pdrgi.head(1)

# Preselection of Leclercq IDs by lat/lon rectangle

In [ ]:
lon_range = [0, 20]
lat_range = [40, 50]

In [ ]:
alps_ids = lec[(lec.lon >= lon_range[0]) & (lec.lon <= lon_range[1]) &
               (lec.lat >= lat_range[0]) & (lec.lat <= lat_range[1])].ID.values

In [ ]:
alps_ids

# Create an overview plot with ALL glaciers

In [ ]:
curr = 1  # current plot
total = len(alps_ids)  # total number of plots ()

dummyline = plt.Line2D((0,1),(0,0), color='w', linewidth=0, linestyle='')  # for dummy 'category' entry

with PdfPages(os.path.join(output_dir, 'Leclercq_Glaciers_Alps_ALL.pdf')) as pdf:
    for gid in alps_ids:
        glacier = lec[lec.ID == gid].iloc[0]
        lon, lat = glacier.lon, glacier.lat
        pdrgi['DIST'] = haversine(lon, lat, pdrgi.CenLon.values, pdrgi.CenLat.values)
        pda['DIST'] = haversine(lon, lat, pda.LONGITUDE.values, pda.LATITUDE.values)
        
        sortrgi = pdrgi.sort(columns='DIST')
        sortpda = pda.sort(columns='DIST')
        
        # For GoogleMap we need a lon lat range to generate the map
        mmlon = [lon, lon]
        mmlat = [lat, lat]
        
        for i in np.arange(0,5):
            rgig = sortrgi.iloc[i]
            pdag = sortpda.iloc[i]
            # In case the glacier is a MultiPolygon we account for this here:
            if rgig.geometry.type == 'Polygon':
                x, y = rgig.geometry.exterior.xy
            elif rgig.geometry.type == 'MultiPolygon':
                # buffer is necessary as some multi-polygons are self-intersecting
                allparts = [p.buffer(0) for p in rgig.geometry] 
                rgig.geometry = shapely.ops.cascaded_union(allparts)
                x, y = rgig.geometry.exterior.xy
            
            # extend map extent by RGI geometries
            mmlon = [np.min(np.append(mmlon, x)), np.max(np.append(mmlon, x))]
            mmlat = [np.min(np.append(mmlat, y)), np.max(np.append(mmlat, y))]
            
            # extend map by WGMS lat/lon (in case they are outside the bounds)
            mmlon = [np.min(np.append(mmlon, pdag.LONGITUDE)), np.max(np.append(mmlon, pdag.LONGITUDE))]
            mmlat = [np.min(np.append(mmlat, pdag.LATITUDE)), np.max(np.append(mmlat, pdag.LATITUDE))]
        
        # Make a local map where to plot the polygons
        local = datasets.GoogleVisibleMap(x=mmlon, y=mmlat) # also possible:  maptype='terrain'
        local_map = Map(local.grid, countries=False, nx=640)
        local_map.set_lonlat_countours()
        
        
        # Prepare the figure
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        ax1.set_title('WGMS: ' + str(sortpda.iloc[0].WGMS_ID) + '-' + sortpda.iloc[0].POLITICAL_UNIT + '-' + 
                      sortpda.iloc[0].NAME + ' --- LEC: ' + str(gid) + ', ' +str(glacier['name']) +', '+ 
                      glacier['area (km2)']+'km2') # 'glacier['name']' is absolutley necessary this way, otherwise Python gets confused with glacier.Name (despite case sensitivity!)
        ax2.set_title("Smallest Haversine distance: %.2f m" % (haversine(lon, lat, sortrgi.iloc[0].CenLon, sortrgi.iloc[0].CenLat)))

        # Plot glaciers
        colors = ['red', 'orange', 'green', 'blue', 'purple', 'magenta']
        for i in np.arange(0,5):
            rgig = sortrgi.iloc[i]
            pdag = sortpda.iloc[i]
            # In case the glacier is a MultiPolygon we account for this here:
            if rgig.geometry.type == 'Polygon':
                x, y = rgig.geometry.exterior.xy
            elif rgig.geometry.type == 'MultiPolygon':
                # buffer is necessary as some multi-polygons are self-intersecting
                allparts = [p.buffer(0) for p in rgig.geometry] 
                rgig.geometry = shapely.ops.cascaded_union(allparts)
                x, y = rgig.geometry.exterior.xy
            
            #  print centroid of matching RGIglacier
            if i == 0:
                # Plot the RGI point
                local_map.set_geometry(shpg.Point(rgig.CenLon, rgig.CenLat), edgecolor='k', marker='x', linewidth=4, markersize=100, zorder=50, text='RGI')
                # Plot the WGMS point
                local_map.set_geometry(shpg.Point(pdag.LONGITUDE, pdag.LATITUDE), color='g', marker='x', linewidth=4, markersize=100, zorder=50, text='WGMS')
            
            # RGI polygon label
            if rgig.Name == None:
                plabel =  str(rgig.RGIId)+'\n'+str(rgig.Area)+'km2'
            else:
                plabel =  str(rgig.RGIId)+'\n'+str(rgig.Area)+'km2\n'+str(rgig.Name)
            
            local_map.set_geometry(rgig.geometry.exterior, color=colors[i], linewidth=3, label=plabel) #adjusted for RGI 5.0
        # Plot the Leclercq Point
        local_map.set_geometry(shpg.Point(lon, lat), c='k', marker='x', linewidth=4, markersize=100, zorder=51, text='LEC') #again adjusted fpr RGI 5.0
        
        
        local_map.set_rgb(local.get_vardata())
        local_map.visualize(ax=ax1, addcbar=False)

        local = datasets.GoogleVisibleMap(x=mmlon, y=mmlat, maptype='terrain')
        local_map.set_rgb(local.get_vardata())
        local_map.visualize(ax=ax2, addcbar=False)
        plt.subplots_adjust(left=0.04, right=0.80, top=0.94, bottom=0.07)
        handles, labels = ax1.get_legend_handles_labels()
        plt.legend(bbox_to_anchor=(1.02, 1.), fontsize=18, loc=2, borderaxespad=0, frameon=False, numpoints=1, scatterpoints=1)
        
        pdf.savefig(fig)
        plt.close()
        
        if curr % 5 == 0:
            print "%s / %s plots done." % (curr, total)
        curr += 1

# Try an automated linking to WGMS and RGI

In [ ]:
manual_links = lec.copy()
manual_links = manual_links[manual_links.ID.isin(alps_ids)]
manual_links['WGMS_ID'] = np.nan
manual_links['RGI_ID'] = np.nan

for gid in alps_ids:
    glacier = lec[lec.ID == gid].iloc[0]
    lon, lat = glacier.lon, glacier.lat
    pdrgi['DIST'] = haversine(lon, lat, pdrgi.CenLon.values, pdrgi.CenLat.values)
    pda['DIST'] = haversine(lon, lat, pda.LONGITUDE.values, pda.LATITUDE.values)

    sortrgi = pdrgi.sort(columns='DIST')
    sortpda = pda.sort(columns='DIST')

    rgig = sortrgi.iloc[0]
    pdag = sortpda.iloc[0]
    # In case the glacier is a MultiPolygon we account for this here:
    if rgig.geometry.type == 'Polygon':
        x, y = rgig.geometry.exterior.xy
    elif rgig.geometry.type == 'MultiPolygon':
        # buffer is necessary as some multi-polygons are self-intersecting
        allparts = [p.buffer(0) for p in rgig.geometry] 
        rgig.geometry = shapely.ops.cascaded_union(allparts)
        x, y = rgig.geometry.exterior.xy
    
    manual_links.WGMS_ID[manual_links.ID == gid] = pdag.WGMS_ID
    print gid, pdag.WGMS_ID, manual_links[manual_links.ID == gid].WGMS_ID
    manual_links.RGI_ID[manual_links.ID == gid] = rgig.RGIId
        
manual_links.to_csv(os.path.join(output_dir,'Automated_links_Leclercq_to_RGI_WGMS_ALPS.csv'))